Large Waste Items:
- Number of detections: 65
- Average confidence: 0.427
- Maximum confidence: 0.912
- Minimum confidence: 0.252

PP bag:
- Number of detections: 44
- Average confidence: 0.557
- Maximum confidence: 0.943
- Minimum confidence: 0.270

General Waste:
- Number of detections: 163
- Average confidence: 0.426
- Maximum confidence: 0.741
- Minimum confidence: 0.250

CleanNet:
- Number of detections: 119
- Average confidence: 0.745
- Maximum confidence: 0.959
- Minimum confidence: 0.265

이걸 기준으로 평균보다 아래는 Red바운딩 박스 이상은 Green 바운딩 박스를 표출한다.

iou적용 및 위도경도 csv로 내려받을 수 있도록 수정하였다.

In [3]:
# Cell 1: 필요한 라이브러리 임포트
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os
import json
from pathlib import Path
import numpy as np
from time import time
import pandas as pd
from datetime import datetime

In [4]:
# Cell 2: IoU 관련 함수 정의
def calculate_iou(box1, box2):
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    
    x1_i = max(x1_1, x1_2)
    y1_i = max(y1_1, y1_2)
    x2_i = min(x2_1, x2_2)
    y2_i = min(y2_1, y2_2)
    
    if x2_i < x1_i or y2_i < y1_i:
        return 0.0
    
    intersection = (x2_i - x1_i) * (y2_i - y1_i)
    area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0

def non_max_suppression(boxes, scores, classes, iou_threshold=0.5):
    if len(boxes) == 0:
        return [], [], []
        
    selected_indices = []
    indices = np.argsort(scores)[::-1]
    
    while len(indices) > 0:
        current_idx = indices[0]
        selected_indices.append(current_idx)
        
        if len(indices) == 1:
            break
            
        ious = [calculate_iou(boxes[current_idx], boxes[idx]) for idx in indices[1:]]
        indices = indices[1:][np.array(ious) < iou_threshold]
    
    return [boxes[i] for i in selected_indices], [scores[i] for i in selected_indices], [classes[i] for i in selected_indices]

In [5]:
# Cell 3: 카테고리 매핑 정의
category_mapping = {
    # 대형폐기물 (Large Waste Items)
    'arcade machine': 'Large Waste Items',
    'Audio': 'Large Waste Items',
    'Computer': 'Large Waste Items',
    'fax machine': 'Large Waste Items',
    'Main unit': 'Large Waste Items',
    'Monitor': 'Large Waste Items',
    'Printer': 'Large Waste Items',
    'sewing machine': 'Large Waste Items',
    'Speaker': 'Large Waste Items',
    'typewriter': 'Large Waste Items',
    'Vacuum cleaner': 'Large Waste Items',
    'Video player': 'Large Waste Items',
    'Bathtub': 'Large Waste Items',
    'Sink': 'Large Waste Items',
    'Kitchen sink': 'Large Waste Items',
    'Toilet bowl': 'Large Waste Items',
    'Bed': 'Large Waste Items',
    'Bookcase': 'Large Waste Items',
    'Bookstand': 'Large Waste Items',
    'Cabinet': 'Large Waste Items',
    'chair': 'Large Waste Items',
    'Cupboard': 'Large Waste Items',
    'Desk': 'Large Waste Items',
    'Dining table': 'Large Waste Items',
    'Display cabinet': 'Large Waste Items',
    'Display stand': 'Large Waste Items',
    'Drawer unit': 'Large Waste Items',
    'Shoe rack': 'Large Waste Items',
    'Small cabinet': 'Large Waste Items',
    'Sofa': 'Large Waste Items',
    'Table': 'Large Waste Items',
    'TV stand': 'Large Waste Items',
    'Vanity table': 'Large Waste Items',
    'Wardrobe': 'Large Waste Items',
    'Air conditioner': 'Large Waste Items',
    'Air purifier': 'Large Waste Items',
    'dish dryer': 'Large Waste Items',
    'Electric rice cooker': 'Large Waste Items',
    'Fan': 'Large Waste Items',
    'Gas oven range': 'Large Waste Items',
    'Heater': 'Large Waste Items',
    'Humidifier': 'Large Waste Items',
    'Microwave': 'Large Waste Items',
    'refrigerator': 'Large Waste Items',
    'Spin dryer': 'Large Waste Items',
    'TV': 'Large Waste Items',
    'Washing machine': 'Large Waste Items',
    'Aquarium': 'Large Waste Items',
    'Bamboo mat': 'Large Waste Items',
    'Bedding items': 'Large Waste Items',
    'bicycle': 'Large Waste Items',
    'Carpet': 'Large Waste Items',
    'Clothes drying rack': 'Large Waste Items',
    'Coat rack': 'Large Waste Items',
    'Door panel': 'Large Waste Items',
    'Earthenware jar': 'Large Waste Items',
    'Floor covering': 'Large Waste Items',
    'Frame': 'Large Waste Items',
    'lumber': 'Large Waste Items',
    'Mannequin': 'Large Waste Items',
    'Mat': 'Large Waste Items',
    'Piano': 'Large Waste Items',
    'Rice storage container': 'Large Waste Items',
    'Signboard': 'Large Waste Items',
    'Stroller': 'Large Waste Items',
    'Wall clock': 'Large Waste Items',
    'Water tank': 'Large Waste Items',
    'audio cabinet': 'Large Waste Items',
    'suitcase': 'Large Waste Items',
    
    # 기타 카테고리
    'PP bag': 'PP bag',
    'General waste bag': 'General Waste',
    'waste pile': 'General Waste',
    'CleanNet': 'CleanNet',
    'General Waste': 'General Waste'
}

In [6]:
# Cell 4: 바운딩 박스 색상 및 시간 계산 함수
def get_bbox_color(class_name, confidence):
    mapped_class = category_mapping.get(class_name, class_name)
    
    if mapped_class == 'Large Waste Items':
        return (0, 255, 0) if confidence > 0.4 else (0, 0, 255)
    elif mapped_class == 'PP bag':
        return (0, 255, 0) if confidence > 0.5 else (0, 0, 255)
    else:
        return (255, 255, 255)

def calculate_average_times(process_times):
    avg_times = {
        'preprocess': sum(t['preprocess'] for t in process_times) / len(process_times),
        'inference': sum(t['inference'] for t in process_times) / len(process_times),
        'postprocess': sum(t['postprocess'] for t in process_times) / len(process_times)
    }
    
    return avg_times

In [7]:
# Cell 5: 파일명 파싱 함수
def parse_location_from_filename(filename):
    parts = filename.split('-')
    if len(parts) >= 5:
        lat = float(f"{parts[1]}.{parts[2]}")
        lon = float(f"{parts[3]}.{parts[4].split('_')[0]}")
        return lat, lon
    return None, None

def parse_time_from_filename(filename):
    time_str = filename[:12]
    try:
        return datetime.strptime(time_str, '%Y%m%d%H%M').strftime('%Y-%m-%d %H:%M:%S')
    except:
        return None

In [8]:
# Cell 6: 객체 감지 함수
def detect_objects(model_path, image_dir, iou_threshold=0.5):
    model = YOLO(model_path)
    output_dir = './runs/detect/predict/'
    os.makedirs(output_dir, exist_ok=True)
    
    image_paths = []
    for ext in ['*.jpg', '*.jpeg', '*.png']:
        image_paths.extend(list(Path(image_dir).glob(ext)))
    
    print(f"Found {len(image_paths)} images in the directory.")
    
    results = []
    process_times = []
    
    for idx, image_path in enumerate(image_paths, 1):
        print(f"\nProcessing image {idx}/{len(image_paths)}: {image_path.name}")
        
        image = cv2.imread(str(image_path))
        if image is None:
            print(f"Error: Could not load image {image_path}")
            continue
            
        result = model.predict(source=str(image_path), save=False, device='cpu')[0]
        
        process_times.append({
            'preprocess': result.speed['preprocess'],
            'inference': result.speed['inference'],
            'postprocess': result.speed['postprocess']
        })
        
        boxes = result.boxes.xyxy.cpu().numpy()
        scores = result.boxes.conf.cpu().numpy()
        class_ids = result.boxes.cls.cpu().numpy()
        class_names = [result.names[int(cls_id)] for cls_id in class_ids]
        
        filtered_boxes, filtered_scores, filtered_classes = non_max_suppression(
            boxes, scores, class_names, iou_threshold
        )
        
        for box, confidence, class_name in zip(filtered_boxes, filtered_scores, filtered_classes):
            x1, y1, x2, y2 = box.astype(int)
            try:
                color = get_bbox_color(class_name, confidence)
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
                label = f'{class_name} {confidence:.2f}'
                cv2.putText(image, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            except Exception as e:
                print(f"Error processing detection for class {class_name}: {e}")
                continue
        
        output_path = os.path.join(output_dir, f'{Path(image_path).stem}_detected{Path(image_path).suffix}')
        cv2.imwrite(output_path, image)
        
        results.append({
            'image_name': image_path.name,
            'detections': [
                {'class': cls, 'confidence': conf, 'box': box.tolist()}
                for cls, conf, box in zip(filtered_classes, filtered_scores, filtered_boxes)
            ]
        })
    
    avg_times = calculate_average_times(process_times)
    return results, avg_times

In [9]:
def save_results(results, avg_times, output_path):
    # CSV 데이터를 저장할 리스트
    csv_data = []
    
    for result in results:
        image_name = result['image_name']
        lat, lon = parse_location_from_filename(image_name)
        time = parse_time_from_filename(image_name)
        
        # 각 이미지의 각 객체별 정보 저장
        detections_by_type = {}
        for detection in result['detections']:
            obj_type = category_mapping.get(detection['class'], detection['class'])
            if obj_type not in detections_by_type:
                detections_by_type[obj_type] = {
                    'count': 0,
                    'total_score': 0,
                }
            detections_by_type[obj_type]['count'] += 1
            detections_by_type[obj_type]['total_score'] += detection['confidence']
        
        # 이미지 파일명 생성 (detected 접미사 추가)
        detected_image_name = f"{os.path.splitext(image_name)[0]}_detected.jpg"
        
        # 각 객체 타입별로 CSV 행 생성
        for obj_type, stats in detections_by_type.items():
            avg_score = stats['total_score'] / stats['count']
            csv_data.append({
                'Latitude': lat,
                'Longitude': lon,
                'Type': obj_type,
                'Count': stats['count'],
                'Time': time,
                'Image': detected_image_name,  # 수정된 부분: 파일명만 저장
                'Score': round(avg_score, 3)
            })
    
    # DataFrame 생성 및 CSV 저장
    df = pd.DataFrame(csv_data)
    csv_output_path = output_path + '.csv'
    df.to_csv(csv_output_path, index=False, encoding='utf-8-sig')
    
    # 요약 통계 반환
    summary = {
        "total_images": len(results),
        "positive_count": sum(1 for result in results if any(
            category_mapping.get(det['class'], det['class']) == 'Large Waste Items'
            for det in result['detections']
        )),
        "negative_count": sum(1 for result in results if not any(
            category_mapping.get(det['class'], det['class']) == 'Large Waste Items'
            for det in result['detections']
        ))
    }
    
    return summary

In [10]:
# Cell 8: 메인 실행 코드
def main():
    model_path = 'results_20241124_155336/waste_detection_model/weights/best.pt'
    image_dir = 'test/images'
    output_path = './test/detection_results'  # .csv will be added automatically
    iou_threshold = 0.5
    
    print("Starting object detection with IoU filtering...")
    results, avg_times = detect_objects(model_path, image_dir, iou_threshold)
    
    print("\nSaving results to CSV...")
    summary = save_results(results, avg_times, output_path)
    
    print("\nProcessing Summary:")
    print(f"Total images processed: {summary['total_images']}")
    print(f"Positive detections: {summary['positive_count']}")
    print(f"Negative detections: {summary['negative_count']}")
    print(f"\nAverage Processing Times:")
    print(f"Preprocess: {avg_times['preprocess']:.2f}ms")
    print(f"Inference: {avg_times['inference']:.2f}ms")
    print(f"Postprocess: {avg_times['postprocess']:.2f}ms")
    print(f"Total Average: {sum(avg_times.values()):.2f}ms per image")

if __name__ == "__main__":
    main()

Starting object detection with IoU filtering...
Found 76 images in the directory.

Processing image 1/76: 202411061313-36-307488-127-352668_jpg.rf.2debbe814df92503a0079be859650cc0.jpg

image 1/1 c:\Users\yimsebin\livingLab\demo\model\test\images\202411061313-36-307488-127-352668_jpg.rf.2debbe814df92503a0079be859650cc0.jpg: 640x640 1 General Waste, 2 CleanNets, 59.4ms
Speed: 4.0ms preprocess, 59.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

Processing image 2/76: 202411061319-36-307974-127-355197_jpg.rf.40e66b47ea454f90ee055c740410cc1c.jpg

image 1/1 c:\Users\yimsebin\livingLab\demo\model\test\images\202411061319-36-307974-127-355197_jpg.rf.40e66b47ea454f90ee055c740410cc1c.jpg: 640x640 3 General Wastes, 2 CleanNets, 27.2ms
Speed: 2.1ms preprocess, 27.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

Processing image 3/76: 202411061322-36-307550-127-354681_jpg.rf.e09f59bc4cc8915a9da9c47c91f306f0.jpg

image 1/1 c:\Users\yimsebin\livingLab\demo\m